In [1]:
# !pip install transforms

In [2]:
# 1_extract_and_store_features_final_fixed.py

import os
import numpy as np
import torch
from model_init import load_model, preprocess_image, get_normalized_features

# Paths
reference_folder = "../60_images_of_6_cows/db-images2"  # folder with cattle subfolders
features_output_file = "reference_features.npy"
filenames_output_file = "reference_filenames.npy"

# Device setup
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

# Load model
model = load_model("best_model-3.pth", device)

# Extract and store features
final_features = []
final_filenames = []

# Process each subfolder
for subfolder in os.listdir(reference_folder):
    if subfolder.startswith("cropped_img"):
        subfolder_path = os.path.join(reference_folder, subfolder)
        if not os.path.isdir(subfolder_path):
            continue  # Skip if not a folder
        
        cattle_features = []

        # Extract features for all images inside this subfolder
        for filename in os.listdir(subfolder_path):
            if filename.lower().endswith((".jpg", ".png", ".jpeg")):
                image_path = os.path.join(subfolder_path, filename)
                image_tensor = preprocess_image(image_path)
                feature = get_normalized_features(model, image_tensor, device)
                cattle_features.append(feature.squeeze())  # convert to numpy

        if len(cattle_features) == 0:
            print(f"No valid images found in {subfolder}. Skipping...")
            continue
        
        cattle_features = np.stack(cattle_features)  # shape: (num_images, 1536)

        # Calculate mean feature (no random sampling)
        mean_feature = np.mean(cattle_features, axis=0)  # shape: (1536,)

        # Optional: normalize the mean feature
        mean_feature /= np.linalg.norm(mean_feature) + 1e-8

        # Store
        final_features.append(mean_feature)
        final_filenames.append(subfolder)  # using subfolder name as cattle id

        print(f"Processed cattle ID: {subfolder}, Feature shape: {mean_feature.shape}")

# Save all features and corresponding cattle IDs
final_features = np.stack(final_features)  # shape: (num_cattles, 1536)
np.save(features_output_file, final_features)
np.save(filenames_output_file, np.array(final_filenames))

print("\n✅ Saved all cattle features and filenames successfully.")


/Users/niteshkumar/conda/envs/tf_env/lib/python3.9/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


Using device: mps
Processed cattle ID: cropped_img207, Feature shape: (1536,)
Processed cattle ID: cropped_img209, Feature shape: (1536,)
Processed cattle ID: cropped_img214, Feature shape: (1536,)
Processed cattle ID: cropped_img217, Feature shape: (1536,)
Processed cattle ID: cropped_img221, Feature shape: (1536,)
Processed cattle ID: cropped_img216, Feature shape: (1536,)

✅ Saved all cattle features and filenames successfully.
